# Vector Store Implementation

The following code is an implementation made for the final exam of the Information Retrival course.

**Author:** *Jacopo Zacchigna*

---

The notebook is an implementation of a vector store.
The code is structured in multiple class:

- Index
- VectorStore

And there is also the implementation of additional classes that are helpfull to load the data and retrive interesting informations:

- TextLoader
- DirectoryReader
- TextSplitter

---

##### The text for the RAG demo is taken from:

- Grokking Paper: https://arxiv.org/pdf/2201.02177.pdf
- Attention Is All You Need: https://arxiv.org/pdf/1706.03762.pdf

### Imports external libraries

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import pickle
import os

## Index (Helper class)

Implementation of an Helper class index which is going to be used in my vector Store

- **Add items:** to add all of the vectors with the relative indices to the stored_vectors dictionary
- **knn_query:** to get the `top_n` most similar vectors inside a vector store with the relative
- **_cosine_similarity:** helper function to compute the cosine similarity

In [2]:
class Index:
    def __init__(self, dim=None):
        self.dim = dim
        
        # Dictionary to store the vectors
        self.stored_vectors = {}

    def add_items(self, vectors, vectors_id: int):
        """
        Update the indexing structure for the vector store
        """
        for vector_id, vector in zip(vectors_id, vectors):
            if vector.shape != (self.dim,):
                raise ValueError("Vectors must have shape (dim,)")
            self.stored_vectors[vector_id] = vector

    def knn_query(self, query_vector: np.ndarray, top_n: int = 5):
        """
        Find the top n similar vectors to the query vector using cosine similarity.

        Args:
            query_vector (numpy.ndarray): The query vector.
            top_n (int): The number of top similar vectors to return.

        Returns:
            A tuple of two numpy arrays: the first array contains the indices of the top n similar vectors,
            and the second array contains the corresponding cosine similarity scores.
        """
        similarities = [(index, self._cosine_similarity(query_vector, vector)) for index, vector in self.stored_vectors.items()]

        # Sort based on the similarity (second element of the vector) and take the first top_n elements
        # Then unpack it into indices and distances
        top_n_indices, top_n_similarities = zip(*sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n])

        return top_n_indices, top_n_similarities
        
    def _cosine_similarity(self, query_vector, vector) -> float:
        """
        Compute the similarity between two vectors

        Args:
            query_vector (numpy.ndarray): The query vector
            vector (numpy.ndarray): The vector to compare

        Returns:
            The dot product of the vectors, normalized by the product of their norms
        """

        dot_product = np.dot(query_vector, vector)
        
        query_vector_norm = np.linalg.norm(query_vector)
        vector_norm = np.linalg.norm(vector)

        # Return the similarity
        return dot_product / (query_vector_norm * vector_norm)

## Vector Store class

This is the main part of the code which implements the vector store.

*It focueses on implementing the following function with some additional functionality and some basic error handling:*

- **_load_vector_store:** loads the index and sentences

- **save_vector_store:** saves the index and sentences to the specified directory

- **create_vector_store:** adds vectors to the vector store

- **update_vector_store:** updates the existing vector store with new vectors

- **delete_vector_store:** deletes a persistent vector store

- **get_similar_vectors:** finds similar vectors to the query vector based on cosine similarity

In [3]:
class VectorStore:
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", persist=True, persist_path="vector_store"):
        self.persist = persist
        self.persist_path = persist_path
        
        # Initialize our index our index
        self.model = SentenceTransformer(model_name, trust_remote_code=True)
        
        # Counter to then store the ids of vectors
        self.id_counter = 0

        # Dictionary to store sentences corresponding to vectors
        self.sentences = {}

    def _load_vector_store(self):
        index_file = os.path.join(self.persist_path, "index.pkl")
        sentences_file = os.path.join(self.persist_path, "sentences.pkl")

        if not os.path.exists(index_file) or not os.path.exists(sentences_file):
            raise FileNotFoundError("Index and sentences files not found in the specified directory.")

        with open(index_file, "rb") as f:
            self.index = pickle.load(f)
        with open(sentences_file, "rb") as f:
            self.sentences = pickle.load(f)

        return self.index, self.sentences

    def _save_vector_store(self):
        # Create the directory if it doesn't exist
        os.makedirs(self.persist_path, exist_ok=True)

        # Serialize and save the index
        with open(os.path.join(self.persist_path, "index.pkl"), "wb") as f:
            pickle.dump(self.index, f)

        # Serialize and save the sentences
        with open(os.path.join(self.persist_path, "sentences.pkl"), "wb") as f:
            pickle.dump(self.sentences, f)

    def create_vector_store(self, documents):
        # Get the embeddings
        embeddings = self.model.encode(documents)
        self.embeddings_dimension = len(embeddings[0])

        # Create the index
        self.index = Index(dim=self.embeddings_dimension)

        # Create a dictionary with the documents and the relative embeddings
        new_documents_embeddings = {documents[i]: embeddings[i] for i in range(len(documents))}
        
        try:
            vectors = []
            ids = []
            
            for sentence, vector in new_documents_embeddings.items():
                # Append the new vector
                vectors.append(vector)
                # Assign a unique integer id to every vector
                ids.append(self.id_counter)
                # Store the sentence
                self.sentences[self.id_counter] = sentence
                # Increment the counter for the next vector
                self.id_counter += 1
                
            # Adding the items to the index
            self.index.add_items(vectors, ids)

            if self.persist:
                self._save_vector_store()

            print("\033[32mVector store created successfully\033[0m", end="\n\n")
        except Exception as e:
            raise e

    def update_vector_store(self, documents):
        """
        Update the existing vector store with new documents

        documents: List of documents to add to my vector store
        """
        embeddings = self.model.encode(documents)
        new_documents_embeddings = {documents[i]: embeddings[i] for i in range(len(documents))}

        try:
            # Load existing index and sentences
            self.index, self.sentences = self._load_vector_store()

            # Update the id counter
            self.id_counter = max(self.sentences.keys()) + 1

            # Add new vectors to the index and sentences
            vectors = []
            ids = []
            for sentence, vector in new_documents_embeddings.items():
                vectors.append(vector)
                ids.append(self.id_counter)
                self.sentences[self.id_counter] = sentence
                self.id_counter += 1

            # Adding the vectors, index to the our index
            self.index.add_items(vectors, ids)

            print("\033[32mVector store updated successfully\033[0m", end="\n\n")
        except Exception as e:
            raise e

    def delete_vector_store(self) -> None:
        """
        Delete a persistent vector store that was craeted
        """
        try:
            # Check if the directory exists
            if os.path.exists(self.persist_path):
                # Delete index and sentences files
                os.remove(os.path.join(self.persist_path, "index.pkl"))
                os.remove(os.path.join(self.persist_path, "sentences.pkl"))
                os.rmdir(self.persist_path)
                print("\033[32mVector store deleted successfully\033[0m", end="\n\n")
            else:
                print("Vector store does not exist", end="\n\n")
        except Exception as e:
            raise e

    def query_similar_vectors(self, query: str, top_n=5):
        """
        Find similar vectors to the query

        Args:
            query (str): The query that is going to be searched for inside my vector store
            num_results (int): The number of similar vectors to return

        Returns:
            A list of tuples, each containing a document and its similarity to the query vector
        """
        if self.persist:
            # Load existing index and sentences
            self._load_vector_store()

        # Use the same model to encode the query
        query_vector = self.model.encode(query)
        
        # Querry for the top_n most similar vectors to my querry vector
        labels, distances = self.index.knn_query(query_vector, top_n=top_n)

        # Return the most similar documents in a list of tuples with (sentence, similarity_score)
        return [(self.sentences[label], distance) for label, distance in zip(labels, distances)]
        
    def print_similar_vectors(self, similar_vectors) -> None:
        """
        Helper function to print the most similar vector with the relative similarity score in a nice way
        """
        print("\033[1mSimilar Text Retrived:\033[0m")
        print("___________________________________\n")
        for sentence, similarity_score in similar_vectors:
            print("\033[1m- Retrieved Text:\033[0m", sentence)
            print("\033[1m    Similarity Score:\033[0m", similarity_score)
            print()

## Demo of The vector store

Using nomic embed for the demo and a custom index

#### TextSplitter and Retriver class

Implementation of the text splitter with different options

In [4]:
class TextLoader:
    def __init__(self, data_path):
        self.data_path = data_path

    def load_data(self):
        # Load the data from a CSV file
        return pd.read_csv(self.data_path, delimiter=";")

    def split_data(self, split_ratio=0.8, random_state=1337):
        # Load the data
        data = self.load_data()

        # Split them in two splits
        data_1 = data.sample(frac=split_ratio, random_state=random_state)
        data_2 = data.drop(data_1.index)
        
        return data_1["text"].tolist(), data_2["text"].tolist()

### Vector Store Creatoion

Creation of the vector store, and simple query

In [5]:
# Get the raw text
# Split to then use it to update the vector store
data_1, data_2 = TextLoader('data/sample.csv').split_data()

# Create the vecotr store from documents
db = VectorStore(model_name="nomic-ai/nomic-embed-text-v1.5", persist=True, persist_path="demo")

# Create the vector store
db.create_vector_store(data_1)

# Define a querry and searcah for it in my vector stor
query = "I want to buy a car"
similar_vectors = db.query_similar_vectors(query)

db.print_similar_vectors(similar_vectors)

<All keys matched successfully>


Vector store created successfully

Similar Text Retrived:
___________________________________

- Retrieved Text: A vintage car gleamed in the sunlight, its polished chrome catching the eye of passersby.
    Similarity Score: 0.5708912

- Retrieved Text: The sleek, silver sports car raced down the winding mountain road, its engine roaring with power.
    Similarity Score: 0.52294475

- Retrieved Text: A family sedan cruised along the highway, its occupants singing along to their favorite songs on the radio.
    Similarity Score: 0.5017636

- Retrieved Text: The Harley-Davidson motorcycle rumbled to life, its deep, throaty growl announcing its presence on the road.
    Similarity Score: 0.491886

- Retrieved Text: The sound of engines filled the air, a symphony of power and speed that echoed through the streets.
    Similarity Score: 0.47884065



### Showcase The Vector Store Update

In [6]:
# Update the vector store
db.update_vector_store(data_2)

# Query the vector store
query = "I want to buy a cycle"

similar_vectors = db.query_similar_vectors(query, top_n=3)
db.print_similar_vectors(similar_vectors)

Vector store updated successfully

Similar Text Retrived:
___________________________________

- Retrieved Text: And as the sun rises once again, the cycle begins anew, a testament to the beauty and resilience of life.
    Similarity Score: 0.53241175

- Retrieved Text: The Harley-Davidson motorcycle rumbled to life, its deep, throaty growl announcing its presence on the road.
    Similarity Score: 0.5212596

- Retrieved Text: A vintage car gleamed in the sunlight, its polished chrome catching the eye of passersby.
    Similarity Score: 0.4940513



#### Deleting the Vector Store

In [7]:
# Delete saved vector store
db.delete_vector_store()

Vector store deleted successfully



## Showcase RAG Application

### Additional Imports

In [8]:
from pypdf import PdfReader
import ollama

### DirectoryReader, TextSplitter

In [25]:
class DirectoryReader:
    def __init__(self, data_path):
        self.data_path = data_path
        
    def load_data(self):
        # List all files in the data directory
        files = os.listdir(self.data_path)
    
        # Read the contents of each file
        text = ''
        for file in files:
            file_path = os.path.join(self.data_path, file)
            if file.endswith('.pdf'):
                with open(file_path, 'rb') as f:  # Open the file in binary mode
                    pdf = PdfReader(f)  # Create a PdfReader object
                    for page in pdf.pages:
                        text += page.extract_text()
            elif file.endswith('.txt'):
                with open(file_path, 'r') as txt_file:  # Open the file in text mode
                    text += txt_file.read()
            else:
                print(f"File type not supported for: {file}")
    
        return text
        
class CharacterSplitter:
    def __init__(self, chunk_size=100, chunk_overlap=0):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_documents(self, raw_documents):
        # Get the rappresentation for every charcter
        self.data = list(raw_documents)
        self.split_text()
        return self.chunks

    def split_text(self):
        self.chunks = []
        chunk_size = self.chunk_size - self.chunk_overlap  # Adjust chunk size to account for overlap
    
        for i in range(0, len(self.data), chunk_size):
            # Ensure the chunk size doesn't exceed the length of the data
            if i + self.chunk_size > len(self.data):
                chunk = self.data[i:]
            else:
                chunk = self.data[i:i + self.chunk_size]
    
            self.chunks.append("".join(chunk))
    
        # Adjust the last chunk to include the overlap
        if self.chunk_overlap > 0:
            for i in range(1, len(self.chunks)):
                self.chunks[i] = self.chunks[i-1][-self.chunk_overlap:] + self.chunks[i]

In [26]:
# Get the raw text
# Split to then use it to update the vector store
raw_text = DirectoryReader('data/').load_data()

# Craete a tecxt splitter
text_splitter = CharacterSplitter(chunk_size=1000, chunk_overlap=100)

# Split the documents recursivly
data = text_splitter.split_documents(raw_text)

print(data[-2])
print("\n____________________________________________________\n")
print(data[-1])

# Create the vecotr store from documents
db = VectorStore(model_name="nomic-ai/nomic-embed-text-v1.5")

# Create the vector store
db.create_vector_store(data)

# Define a querry and searcah for it in my vector stor
query = "When does grooking happen ?"
similar_vectors = db.query_similar_vectors(query, top_n = 3)
db.print_similar_vectors(similar_vectors)

File type not supported for: sample.csv
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experimebased solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.
∗Equal contribution. Li

<All keys matched successfully>


Vector store created successfully

Similar Text Retrived:
___________________________________

- Retrieved Text: 8 (signiﬁcant with p<0.000014 ). This is suggestive that grokking may only happen after
the network’8 (signiﬁcant with p<0.000014 ). This is suggestive that grokking may only happen after
the network’s parameters are in ﬂatter regions of the loss landscape. It would be valuable for future
work to explore this hypothesis, as well as test other generalization measures.
Figure 7: Networks trained on the S5composition objective appear to only grok in relatively ﬂat
regions of the loss landscape.
10
    Similarity Score: 0.5970477

- Retrieved Text: , long after severely overﬁtting, validation accuracy sometimes suddenly
begins to increase from cha, long after severely overﬁtting, validation accuracy sometimes suddenly
begins to increase from chance level toward perfect generalization. We call this phenomenon
‘grokking’. An example is shown in Figure 1.
• We present the data efﬁc

In [11]:
stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': f"{query}"}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 I'm assuming you meant "grocking," which is a term coined by Cal Newport in his book "Deep Work" to describe the focused, productive study session where one fully engages with a complex concept or skill. Groking typically occurs during dedicated blocks of uninterrupted time, often in a quiet and distraction-free environment. The specific timing can vary depending on the individual's learning style and the complexity of the material being studied. Some people may find that they grok best early in the morning, while others may prefer to study late at night. Ultimately, the goal is to create an environment where you can fully concentrate on the task at hand and immerse yourself in the material until you achieve a deep understanding of it.

In [12]:
# Get only the text and put it all together
context = ' '.join([text for text, _ in similar_vectors])

stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': f"Question: {query} \n Context: {context}"}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 The term "grokking" is not explicitly defined in the given text, but it can be inferred from context that it refers to a phenomenon where neural networks generalize better after being trained on small algorithmically generated datasets. The text suggests that this may occur in relatively flat regions of the loss landscape and that ﬂatness based measures could be predictive of grokking. However, further research is needed to explore this hypothesis.

The process of grokking seems to be related to the networks learning to interpolate a small amount of data while still generalizing to new examples. The authors also mention that they hope to investigate whether any of the measures studied in Jiang et al. (2019) for predicting generalization performance are also predictive of grokking.

It's important to note that the term "grokking" is not a widely used or standard term in machine learning and deep learning literature, so further research would be needed to understand its implications and